In [19]:
GYRO = False
SEQUENCE_LENGTH = 3
SEQUENCE_OVERLAP = 1
BATCH_SIZE = 5
EPOCHS = 20
CATEGORY = "stationary" # "moving" or "stationary"
MODEL_NAME = f"category:{CATEGORY}_epochs:{EPOCHS}_batch:{BATCH_SIZE}_gyro:{GYRO}_window:{SEQUENCE_LENGTH}_overlap:{SEQUENCE_OVERLAP}"


### This code takes in the type of activity, ie "stationary" or "moving" and trains a model for just classifying which phsyical activity of that category the data is  

In [20]:
import file_tagger
import sequence_genrator
import tensorflow as tf
import numpy as np
from keras import layers, Sequential, models
from sklearn.model_selection import train_test_split


STATIONARY_ACTIVITIES = {
    "sitting",
    "standing",
    "lying_down_back",
    "lying_down_stomach",
    "lying_down_right",
    "lying_down_left"
}

STATIONARY_ACTIVITIES_COMBINED = {
    "sitting_or_standing",
    "lying_down_back",
    "lying_down_stomach",
    "lying_down_right",
    "lying_down_left"
}

MOVING_ACTIVITIES = {
    "walking",
    "ascending_stairs",
    "descending_stairs",
    "shuffle_walking",
    "running",
    "misc_movements"
}


DATA_DIRECTORY = "./all_respeck"

activity_dict = {"moving": MOVING_ACTIVITIES,
                       "stationary": STATIONARY_ACTIVITIES
            }

POSSIBLE_ACTIVITIES = activity_dict[CATEGORY]


In [21]:
# split data into training, dev, and test sets
def train_dev_test_split(data, labels, dev_size, test_size):
    # Split the data into training and temporary (dev + test) sets
    train_data, temp_data, train_labels, temp_labels = train_test_split(data, labels, test_size=(dev_size + test_size))
    
    # Split the temporary data into dev and test sets
    dev_data, test_data, dev_labels, test_labels = train_test_split(temp_data, temp_labels, 
                                                                 test_size=(test_size / (dev_size + test_size)))
    
    return train_data, dev_data, test_data, train_labels, dev_labels, test_labels

In [22]:
def generate_training_data(directory, sequence_length, overlap, gyro = GYRO): # if gyro is false, only accelerometer data is used

    tagged_data = []

    # group each csv file into their respective areas
    csv_dictionary = file_tagger.tag_directory(directory)

    # iterates through each activity
    for key in csv_dictionary.keys():
        
        activity = key.split("&")[0]
        if activity in POSSIBLE_ACTIVITIES:

            # iterates through each csv file for the activity 
            for csv_file in csv_dictionary[key]:
                if gyro:
                    sequences = sequence_genrator.generate_sequences_from_file_with_gyroscope(directory + "/" + csv_file, sequence_length, overlap)
                else:
                    sequences = sequence_genrator.generate_sequences_from_file_without_gyroscope(directory + "/" + csv_file, sequence_length, overlap)

                # iterate through each generated sequence
                for sequence in sequences:
                    if activity == "standing" or activity == "sitting":
                        tagged_data.append(("sitting_or_standing", sequence))
                    else:
                        tagged_data.append((activity, sequence))

    print ("there are " + str(len(tagged_data)) + " tagged sequences in the dataset")
    return tagged_data

In [23]:
def train_model_CNN(input_data, labels_encoded, unique_labels, epochs, batch_size, validation_data):
    if GYRO:
        width = 6
    else:
        width = 3
    # Define the CNN model for your specific input shape
    model = Sequential([
        layers.Conv1D(32, 3, activation='relu', input_shape=(SEQUENCE_LENGTH*25, width)),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(128, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Dropout(0.5),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(len(unique_labels), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the CNN model
    model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    return model

In [24]:
tagged_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP)

# Combine all sequences and labels
sequences = [sequence for _, sequence in tagged_sequences]
labels = [label for label, _ in tagged_sequences]

if CATEGORY == "stationary":
    possible_activities = STATIONARY_ACTIVITIES_COMBINED
else:
    possible_activities = POSSIBLE_ACTIVITIES

# encode labels to numbers
sequences = np.array(sequences, dtype=np.float32)
label_to_index = {label: idx for idx, label in enumerate(possible_activities)}
labels_encoded = [label_to_index[label] for label in labels]
labels_encoded = np.array(labels_encoded)

train_data, dev_data, test_data, train_labels, dev_labels, test_labels = train_dev_test_split(sequences, labels_encoded, dev_size=0.1, test_size=0.1) #10% dev, 10% test


# train and save model (CHOOSE BETWEEN CNN AND LSTM)
model = train_model_CNN(train_data, train_labels, possible_activities, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(dev_data, dev_labels)) #batch_size, epochs


# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Save the trained model
#model.save(f"models/Task1/{MODEL_NAME}_{test_accuracy}.keras")

there are 24469 tagged sequences in the dataset
Epoch 1/20
3915/3915 [==============================] - 9s 2ms/step - loss: 0.1435 - accuracy: 0.9685 - val_loss: 0.1031 - val_accuracy: 0.9734
Epoch 2/20
3915/3915 [==============================] - 9s 2ms/step - loss: 0.1025 - accuracy: 0.9749 - val_loss: 0.0930 - val_accuracy: 0.9718
Epoch 3/20
3915/3915 [==============================] - 8s 2ms/step - loss: 0.0945 - accuracy: 0.9768 - val_loss: 0.0683 - val_accuracy: 0.9800
Epoch 4/20
3915/3915 [==============================] - 8s 2ms/step - loss: 0.0846 - accuracy: 0.9788 - val_loss: 0.0696 - val_accuracy: 0.9796
Epoch 5/20
3915/3915 [==============================] - 9s 2ms/step - loss: 0.0803 - accuracy: 0.9786 - val_loss: 0.0908 - val_accuracy: 0.9783
Epoch 6/20
3915/3915 [==============================] - 9s 2ms/step - loss: 0.0793 - accuracy: 0.9784 - val_loss: 0.0762 - val_accuracy: 0.9759
Epoch 7/20
3915/3915 [==============================] - 9s 2ms/step - loss: 0.0791 - acc